In [0]:
%run ./nz_schema

In [0]:
df_nz_2023 = spark.read.format("csv") \
  .schema(schema) \
  .option("header", "true") \
  .load("/FileStore/tables/newzealand_price_mapping.csv")

In [0]:
df_nz_2024 = spark.read.format("csv") \
  .schema(schema) \
  .option("header", "true") \
  .load("/FileStore/tables/Clone_newzealand_price_mapping_2024.csv")

In [0]:
df_nz = df_nz_2023.union(df_nz_2024)

In [0]:
df_price= spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/FileStore/tables/price_invoice_mapping.csv")

def rename_columns(df):
    return df.toDF(*[col.lower().replace(" ", "_") for col in df.columns])

df_price = rename_columns(df_price)

display(df_price)

In [0]:
df_price_month = df_price.withColumn('year',split(df_price.start_of_month,'-').getItem(2))\
            .withColumn('month',split(df_price.start_of_month,'-').getItem(1))\
            .withColumn('day',split(df_price.start_of_month,'-').getItem(0)) \
            .withColumnRenamed('gross_price', 'sales_amount') \
            .withColumn('price_date', to_date(col('start_of_month'), 'dd-MM-yyyy')) \
            .select("product_code", "year", "month", "price_date", "sales_amount")


df_nz_month = df_nz.withColumn('date', to_date(col("date"), "dd-MM-yyyy"))\
             .filter(col("date") <= lit("2024-07-01")) \
             .drop("store_id", "store_code", "sales_amount", "total_sales_amount")

df_month_joined = df_nz_month.join(df_price_month, ["product_code", "year", "month"], "left")

# display(df_month_joined)

In [0]:
df_price_week = df_price.withColumn('year',split(df_price.start_of_week,'-').getItem(0))\
            .withColumn('month',split(df_price.start_of_week,'-').getItem(1))\
            .withColumn('day',split(df_price.start_of_week,'-').getItem(2)) \
            .withColumnRenamed('gross_price', 'sales_amount') \
            .withColumn('price_date', to_date(col('start_of_week'), 'yyyy-MM-dd')) \
            .select("product_code", "year", "month", "price_date", "sales_amount")

df_nz_week = df_nz.withColumn('date', to_date(col("date"), "dd-MM-yyyy"))\
             .filter(col("date") > lit("2024-07-01")) \
             .drop("store_id", "store_code", "sales_amount", "total_sales_amount")

df_week_joined = df_nz_week.join(df_price_week, ["product_code", "year", "month"], "left")

# display(df_week_joined)

**sales_amount <= 2024-07-01**

In [0]:
window = (
    Window.partitionBy("product_code")
    .orderBy(col("year"), col("month"))
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

price_recent_month = df_month_joined.withColumn("recent_sales_amount",
    last(
        when(col("sales_amount").isNotNull() & (col("sales_amount") != 0), col("sales_amount")),
        ignorenulls=True
    ).over(window)
)

# price_recent_month.display()

In [0]:
price_filled_month = price_recent_month.withColumn(
    "sales_amount_new",
    when(col("sales_amount").isNull(), col("recent_sales_amount"))
    .when(col("sales_amount") == 0, col("sales_amount"))
    .otherwise(col("sales_amount"))
)

# display(price_filled_month)


In [0]:
df_1 = price_filled_month.drop("recent_sales_amount", "sales_amount","price_date")\
                    .withColumnRenamed("sales_amount_new", "sales_amount")
# display(df_1)

In [0]:
price_filled_month.select("product_code", "year", "month", "sales_amount", "recent_sales_amount", "sales_amount_new")

df_1_sorted = price_filled_month.groupBy("product_code", "year", "month","sales_amount","recent_sales_amount", "sales_amount_new").count().orderBy("product_code","year","month")

# display(df_1_sorted )

****sales_amount > 2024-07-01****

In [0]:
window = (
    Window.partitionBy("product_code")
    .orderBy(col("year"), col("month"))
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

price_recent_week = df_week_joined.withColumn("recent_sales_amount",
    last(
        when(col("sales_amount").isNotNull() & (col("sales_amount") != 0), col("sales_amount")),
        ignorenulls=True
    ).over(window)
)

# price_recent_week.display()

In [0]:
price_filled_week = price_recent_week.withColumn(
    "sales_amount_new",
    when(col("sales_amount").isNull(), col("recent_sales_amount"))
    .when(col("sales_amount") == 0, col("sales_amount"))
    .otherwise(col("sales_amount"))
)

# display(price_filled_week)


In [0]:
df_2 = price_filled_week.drop("recent_sales_amount", "sales_amount","price_date")\
                    .withColumnRenamed("sales_amount_new", "sales_amount")
# display(df_2)

In [0]:
price_filled_week.select("product_code", "year", "month", "sales_amount", "recent_sales_amount", "sales_amount_new")

df_2_sorted  = price_filled_week.groupBy("product_code", "year", "month","sales_amount","recent_sales_amount", "sales_amount_new").count().orderBy("product_code","year","month")

# display(df_2_sorted)